In [9]:
# Create ngrok.yml configuration file
ngrok_config = """
authtoken: 2gkiMl0AJJZwjXhY678GI9mlwCS_4LAsT9BTcpwnrvGb2JQ7C
tunnels:
  mlflow:
    addr: 5000
    proto: http
"""

with open('ngrok.yml', 'w') as f:
    f.write(ngrok_config)


In [10]:
!pip install mlflow
!pip install pyngrok


In [12]:
from pyngrok import ngrok

# Set your Ngrok authtoken
ngrok.set_auth_token('2gkiMl0AJJZwjXhY678GI9mlwCS_4LAsT9BTcpwnrvGb2JQ7C')

# Start an ngrok tunnel
ngrok_tunnel = ngrok.connect(5000, "http")
print('Public URL:', ngrok_tunnel.public_url)

# Start the MLflow server
get_ipython().system_raw("mlflow ui --port 5000 &")



Public URL: https://3db5-35-199-161-204.ngrok-free.app


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# Set tracking URI to the Ngrok public URL
mlflow.set_tracking_uri(ngrok_tunnel.public_url)

# Load the dataset
file_path = '/content/Student Performance.xlsx'
data = pd.read_excel(file_path)

# Function to clean column names
def clean_column_name(col_name):
    return col_name.strip().replace('#', '').replace(' ', '_').replace(':', '').replace('(', '').replace(')', '')

# Apply the function to all column names
data.columns = [clean_column_name(col) for col in data.columns]

# Remove the second row
data = data.drop(index=1)

# Replace missing values with 0 for numeric columns
for column in data.columns:
    if pd.api.types.is_numeric_dtype(data[column]):
        data[column].fillna(0, inplace=True)

# Ensure 'Final_Score' exists in the cleaned columns
target_variable = 'Final_Score'
if target_variable not in data.columns:
    raise ValueError(f"{target_variable} not found in the columns")

# Assuming 'Final_Score' is the target variable
X = data.drop(columns=[target_variable])
y = data[target_variable]

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant', fill_value=0)

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Create a pipeline that includes the preprocessor and the model
def create_model_pipeline(model):
    return Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

# Initialize models
rf_model = RandomForestRegressor(random_state=42)
gbm_model = GradientBoostingRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42)

# Create pipelines
rf_pipeline = create_model_pipeline(rf_model)
gbm_pipeline = create_model_pipeline(gbm_model)
xgb_pipeline = create_model_pipeline(xgb_model)

# Train the models
rf_pipeline.fit(X_train, y_train)
gbm_pipeline.fit(X_train, y_train)
xgb_pipeline.fit(X_train, y_train)

print("Model Training Complete")

# Predict and evaluate models
def evaluate_model(pipeline, X_test, y_test):
    predictions = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    return mse, r2

rf_mse, rf_r2 = evaluate_model(rf_pipeline, X_test, y_test)
gbm_mse, gbm_r2 = evaluate_model(gbm_pipeline, X_test, y_test)
xgb_mse, xgb_r2 = evaluate_model(xgb_pipeline, X_test, y_test)

print(f"RandomForest MSE: {rf_mse}, R2: {rf_r2}")
print(f"GradientBoosting MSE: {gbm_mse}, R2: {gbm_r2}")
print(f"XGBoost MSE: {xgb_mse}, R2: {xgb_r2}")

# Function to train and log model
def train_and_log_model(pipeline, model_name, X_train, y_train, X_test, y_test):
    try:
        with mlflow.start_run(run_name=model_name):
            pipeline.fit(X_train, y_train)
            predictions = pipeline.predict(X_test)
            mse = mean_squared_error(y_test, predictions)
            r2 = r2_score(y_test, predictions)

            mlflow.log_param("model_name", model_name)
            mlflow.log_metric("mse", mse)
            mlflow.log_metric("r2", r2)

            if model_name == "XGBRegressor":
                mlflow.xgboost.log_model(pipeline.named_steps['model'], artifact_path="model")
            else:
                mlflow.sklearn.log_model(pipeline.named_steps['model'], artifact_path="model")

            print(f"{model_name} MSE: {mse}, R2: {r2}")
    except Exception as e:
        print(f"Failed to log {model_name} model: {e}")

# Log models with MLflow
train_and_log_model(rf_pipeline, "RandomForestRegressor", X_train, y_train, X_test, y_test)
train_and_log_model(gbm_pipeline, "GradientBoostingRegressor", X_train, y_train, X_test, y_test)
train_and_log_model(xgb_pipeline, "XGBRegressor", X_train, y_train, X_test, y_test)

print("Model Tracking Complete")


Model Training Complete
RandomForest MSE: 12.915658909130464, R2: 0.827437876907103
GradientBoosting MSE: 6.8877532607157015, R2: 0.9079748594809287
XGBoost MSE: 6.408346774486543, R2: 0.9143800612341066


/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


RandomForestRegressor MSE: 12.915658909130464, R2: 0.827437876907103
GradientBoostingRegressor MSE: 6.8877532607157015, R2: 0.9079748594809287


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:23:24] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


XGBRegressor MSE: 6.408346774486543, R2: 0.9143800612341066
Model Tracking Complete
